In [3]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from sklearn.utils.class_weight import compute_class_weight
from tqdm import tqdm

# Configuration
data_dir = "../DATA_PREPARE_ATT_06/MixedDataset"
batch_size = 16
num_classes = 8
initial_lr = 1e-3
num_epochs = 20
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define the emotion classes
emotion_classes = ["Anger", "Contempt", "Disgust", "Fear", "Happy", "Neutral", "Sad", "Surprise"]

# Prepare Data Loaders
def prepare_data_loaders(data_dir, batch_size):
    transform_train = transforms.Compose([
        transforms.Grayscale(num_output_channels=3),
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    transform_val_test = transforms.Compose([
        transforms.Grayscale(num_output_channels=3),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    print("Loading datasets...")
    train_dataset = datasets.ImageFolder(os.path.join(data_dir, "train"), transform=transform_train)
    val_dataset = datasets.ImageFolder(os.path.join(data_dir, "val"), transform=transform_val_test)
    test_dataset = datasets.ImageFolder(os.path.join(data_dir, "test"), transform=transform_val_test)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

    return train_loader, val_loader, test_loader, train_dataset

# Compute Class Weights
def compute_weights(train_dataset, num_classes):
    targets = [label for _, label in train_dataset.samples]
    class_weights = compute_class_weight('balanced', classes=np.arange(num_classes), y=targets)
    return torch.tensor(class_weights, dtype=torch.float).to(device)

# Load Teacher Model (ResNet-50)
def load_teacher_model(model_path='teacher_model.pth'):
    if os.path.exists(model_path):
        print("Loading teacher model from saved path...")
        teacher_model = models.resnet50()
        teacher_model.fc = nn.Linear(teacher_model.fc.in_features, num_classes)
        teacher_model.load_state_dict(torch.load(model_path))
    else:
        print("Downloading and saving ResNet-50 as teacher model...")
        teacher_model = models.resnet50(pretrained=True)
        teacher_model.fc = nn.Linear(teacher_model.fc.in_features, num_classes)
        torch.save(teacher_model.state_dict(), model_path)
    teacher_model.eval()  # Set to evaluation mode
    return teacher_model

# Load Student Model (MobileNetV2)
def load_student_model(model_path='student_model.pth'):
    if os.path.exists(model_path):
        print("Loading student model from saved path...")
        student_model = models.mobilenet_v2()
        student_model.classifier[1] = nn.Linear(student_model.last_channel, num_classes)
        student_model.load_state_dict(torch.load(model_path))
    else:
        print("Downloading and saving MobileNetV2 as student model...")
        student_model = models.mobilenet_v2(pretrained=True)
        student_model.classifier[1] = nn.Linear(student_model.last_channel, num_classes)
        torch.save(student_model.state_dict(), model_path)
    return student_model

# Knowledge Distillation Loss
class DistillationLoss(nn.Module):
    def __init__(self, alpha=0.5, temperature=3.0):
        super(DistillationLoss, self).__init__()
        self.alpha = alpha
        self.temperature = temperature
        self.criterion_ce = nn.CrossEntropyLoss()
        self.criterion_kl = nn.KLDivLoss(reduction="batchmean")

    def forward(self, student_outputs, teacher_outputs, labels):
        loss_ce = self.criterion_ce(student_outputs, labels)
        loss_kl = self.criterion_kl(
            torch.log_softmax(student_outputs / self.temperature, dim=1),
            torch.softmax(teacher_outputs / self.temperature, dim=1),
        )
        return self.alpha * loss_ce + (1 - self.alpha) * loss_kl

# Training Loop
def train_model(teacher_model, student_model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs):
    teacher_model.eval()  # Ensure teacher remains frozen
    best_val_accuracy = 0.0

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        student_model.train()
        train_loss = 0.0

        for inputs, labels in tqdm(train_loader, desc="Training"):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            # Forward pass
            student_outputs = student_model(inputs)
            with torch.no_grad():
                teacher_outputs = teacher_model(inputs)  # Teacher predictions

            # Compute loss
            loss = criterion(student_outputs, teacher_outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        train_loss /= len(train_loader)
        print(f"Training Loss: {train_loss:.4f}")

        # Validation
        student_model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for inputs, labels in tqdm(val_loader, desc="Validating"):
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = student_model(inputs)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_accuracy = correct / total
        print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

        # Save best model
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            torch.save(student_model.state_dict(), "mobilenetv2_student_model.pth")
            print(f"New best model saved with accuracy: {val_accuracy * 100:.2f}%")

        scheduler.step()

# Main Script
if __name__ == "__main__":
    # Load Data
    train_loader, val_loader, test_loader, train_dataset = prepare_data_loaders(data_dir, batch_size)

    # Compute Class Weights
    class_weights = compute_weights(train_dataset, num_classes)
    criterion = DistillationLoss(alpha=0.7, temperature=4.0)

    # Load Models
    teacher_model = load_teacher_model().to(device)
    student_model = load_student_model().to(device)

    # Optimizer and Scheduler
    optimizer = optim.AdamW(student_model.parameters(), lr=initial_lr)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10)

    # Train
    train_model(teacher_model, student_model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs)


Using device: cuda
Loading datasets...


c:\Users\Tuf\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Tuf\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\Tuf/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:21<00:00, 4.69MB/s]


c:\Users\Tuf\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



Epoch 1/20


Training:   2%|▏         | 102/6466 [00:52<54:13,  1.96it/s] 


KeyboardInterrupt: 